## Are university towns' housing prices are less affected by the '08 Recession?

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import pandas as pd
import numpy as np

def state_name(row):
    if row['edit'] == True:
        return row[0]
    return ""

def region_name(row):
    if row['edit'] == False:
        return row[0]
    return ""

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data = pd.read_table('data/university_towns.txt', header = None)
    
     # True if it's a state, false it 's a region - find this by looking for [edit] in text
    data['edit'] = data[0].str.contains('\[(edit)\]')
    
    # Make a new row with either just the State or just the Region
    data['State'] = data.apply(lambda row: state_name(row),axis=1)
    data['RegionName'] = data.apply(lambda row: region_name(row),axis=1)
    
    # Drop unneeded rows
    data.drop(['edit',0], axis=1, inplace=True)
    
    # Shitty way to add states to every row under it
    states=[]
    state=""
    for row in data['State']:
        if row != "":
            state = row
        states.append(state)
    data['State'] = pd.Series(states)

    # Remove rows witout a region (just the state)
    data = data[data.RegionName != ""]
    
    # Fix formatting of text
    data['State'] = data.State.str.replace('\[(edit)\]','')
    data['RegionName'] = data.RegionName.str.replace('[ ]\(.*', '')
    return data

get_list_of_university_towns()

/Users/nicholas.corneau/anaconda/envs/data-science/lib/python3.6/site-packages/ipykernel/__main__.py:29: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [4]:
import pandas as pd 

def get_recession():
    df = pd.read_excel('data/gdplev.xls', 
                       parse_cols=[4,6],
                       skiprows=7,
                       names=['Quarter', 'GDP']
                       )
    df = df[df['Quarter'] >= '2000q1'].reset_index().drop('index',axis=1)
    
    recession = False
    recession_end = False
    recession_quarters = []
    for j in range(len(df)-3):
        if df['GDP'][j+2] < df['GDP'][j+1] < df['GDP'][j]:
            recession = True
        if ((df['GDP'][j+3] < df['GDP'][j+2] > df['GDP'][j+1] > df['GDP'][j]) 
            and recession):
                recession=False
                recession_end = True
        if recession:
            recession_quarters.append([df['Quarter'][j], df['GDP'][j]])
    return recession_quarters[1:len(recession_quarters)-1]


def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    recession = get_recession()
    return recession[0][0]

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession = get_recession()
    return recession.pop()[0]

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession = get_recession()
    return min(recession, key=lambda x: x[1])[0]

get_recession_bottom()

'2009q2'

In [7]:
def quarter_labels(year_start, year_end):
    quarters_labels = []
    for i in range(year_end - year_start + 1):
        for j in range(4):
            quarters_labels.append(str(i+year_start) + "q" + str(j+1))
    return list(quarters_labels)    

def group_by_quarter(date):
    l = quarter_labels(1996,2020)
    quarters = pd.date_range(start="1996-04",end='2020-08',freq='3M').strftime('%Y-%m')
    counter = 0
    while (date >= quarters[counter]) and (counter < len(quarters)-1): 
        if quarters[counter] <= date < quarters[counter + 1]:
            return l[counter+1]
        counter += 1

def get_df_and_quarter_data():
    df = pd.read_csv('data/City_Zhvi_AllHomes.csv')
    quarter_data = df.ix[:,'1996-04':'2016-08'].astype(float)
    quarter_data = df.groupby(group_by_quarter, axis=1).apply(lambda x: x.mean(axis=1))
    return df, quarter_data

df, quarter_data = get_df_and_quarter_data()
quarter_data

,1996q2,1996q3,1996q4,1997q1,1997q2,1997q3,1997q4,1998q1,1998q2,1998q3,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
1,154666.666667,154200.000000,154433.333333,156866.666667,158533.333333,159266.666667,162000.000000,166966.666667,171400.000000,175966.666667,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
2,109466.666667,109133.333333,109600.000000,111266.666667,112200.000000,112966.666667,114833.333333,116966.666667,118433.333333,120700.000000,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
3,49833.333333,49366.666667,49466.666667,49600.000000,49733.333333,49733.333333,49866.666667,50300.000000,50800.000000,50666.666667,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
4,87700.000000,88600.000000,89733.333333,91266.666667,92033.333333,93000.000000,94633.333333,96733.333333,98433.333333,100433.333333,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
5,120966.666667,120266.666667,120933.333333,122633.333333,123666.666667,123700.000000,124600.000000,125166.666667,125700.000000,127166.666667,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
6,160733.333333,159600.000000,158633.333333,160633.333333,161966.666667,163333.333333,168200.000000,174633.333333,180333.333333,183700.000000,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
7,NaN,NaN,NaN,68450.000000,67200.000000,67200.000000,70300.000000,73200.000000,72166.666667,71833.333333,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
8,224933.333333,227100.000000,231200.000000,239300.000000,246166.666667,255500.000000,266066.666667,275500.000000,282966.666667,292400.000000,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0
9,77166.666667,76300.000000,76033.333333,76800.000000,77200.000000,77766.666667,78900.000000,80000.000000,80566.666667,80966.666667,...,1.207667e+05,1.217333e+05,1.231667e+05,1.241667e+05,1.269000e+05,1.301333e+05,1.320000e+05,1.339667e+05,1.372000e+05,139900.0


In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''        
    quarter = quarter_data.ix[:,'2000q1':'2016q3']
    dfq = df.ix[:,1:3].join(quarter)
    dfq['State'] = dfq['State'].apply(lambda x: states[x])
    dfq = dfq.set_index(['State','RegionName']).sort_index()
    return dfq

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.666667   70266.666667   70300.000000   
          Brent             92933.333333   94333.333333   96166.666667   
          Brighton                   NaN            NaN            NaN   
          Brookwood         92566.666667   95100.000000   98866.666667   
          Buhl              90800.000000   94600.000000   96500.000000   
          Calera           108933.333333  110366.666667  108000.000000   
          Center Point      80966.666667   81233.333333   81500.000000   
          Centreville       95300.000000   96566.666667   98000.000000   
          Chalkville        94100.000000   94433.333333   94433.333333   
          Chancellor                 NaN            NaN            NaN   
          Chelsea          162066.666667  167033.333333  166900.000000   
          Chickasaw         51200.000000   53666.666667   54933.333333   
          Chunchula         80266.666667   81766.666667   82200.000000   
          Citronelle        64833.333333   66633.333333   68066.666667   
          Clay             120900.000000  122266.666667  123966.666667   
          Coden             62600.000000   64800.000000   66866.666667   
          Coker            118100.000000  120766.666667  118166.666667   
          Concord           78600.000000   78700.000000   80133.333333   
          Cottondale       100833.333333  102633.333333  104766.666667   
...                                  ...            ...            ...   
Wisconsin Vernon           183200.000000  178200.000000  174300.000000   
          Vienna           178033.333333  181533.333333  182433.333333   
          Vinland          119800.000000  126766.666667  134933.333333   
          Wales                      NaN            NaN            NaN   
          Waterford        121200.000000  119433.333333  120200.000000   
          Waukesha         141266.666667  138433.333333  136733.333333   
          Waunakee         187400.000000  191433.333333  192666.666667   
          Waupun            84000.000000   84000.000000   84400.000000   
          Wausau            69566.666667   69466.666667   71033.333333   
          Wayne                      NaN            NaN            NaN   
          West Allis        80933.333333   81933.333333   84066.666667   
          Weston            80166.666667   82700.000000   84166.666667   
          Wheatland        129800.000000  132133.333333  134933.333333   
          Whitelaw          96033.333333  101433.333333  108033.333333   
          Williams Bay     122900.000000  115300.000000  110766.666667   
          Wilson           129033.333333  129366.666667  132433.333333   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        124366.666667  123666.666667  124133.333333   
          Wind Point       149233.333333  145266.666667  140866.666667   
          Wisconsin Dells  100000.000000  103400.000000  105000.000000   
          Wolf River        95166.666667   99333.333333  103800.000000   
          Woodruff                   N

In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivalent to a
    reduced market loss).'''
    start = get_recession_start()
    bottom = get_recession_bottom()
    end = get_recession_end()
    uni_towns = get_list_of_university_towns().reset_index().drop('index', axis=1)
    housing_data = convert_housing_data_to_quarters()
    
    # Group regular housing data and university housing data by state.
    # Clean up all housing data from the start of the recession to the bottom. (2008q3 -> 2009q2)
    r = housing_data.ix[:,start:bottom].reset_index()
    uni_r = r[r.RegionName.isin(uni_towns.RegionName)]
    uni_r_houses_by_state = uni_r.groupby('State').sum()
    houses_r_by_state = r.groupby('State').sum()
    houses_r_by_state = houses_r_by_state[houses_r_by_state.index.isin(uni_r_houses_by_state.index)]
 
    # Clean up all housing data from the right before the recession. (2007q4 -> 2008q3)
    p = housing_data.ix[:,'2007q4':start].reset_index()
    uni = p[p.RegionName.isin(uni_towns.RegionName)]
    uni_houses_by_state = uni.groupby('State').sum()
    houses_by_state = p.groupby('State').sum()
    houses_by_state = houses_by_state[houses_by_state.index.isin(uni_houses_by_state.index)]
    
    # Get the mean of each quarter
    uni_recession_mean = uni_r_houses_by_state.T.mean() 
    houses_recession_mean = houses_r_by_state.T.mean()
    uni_mean = uni_houses_by_state.T.mean() 
    houses_mean = houses_by_state.T.mean()
    
    # If housing prices are lower in the recession, count goes up
    states_uni_affected_by_recession = (uni_recession_mean < uni_mean).sum()
    states_houses_affected_by_recession = (houses_recession_mean < houses_mean).sum()
    
    # Difference is only 2, so there is NOT a big difference between uni town prices and regular towns.
    difference_between_uni_and_houses = abs(states_uni_affected_by_recession - states_houses_affected_by_recession)
    
    p1 = ttest_ind(uni_r_houses_by_state['2008q3'], houses_r_by_state['2008q3'])
    p2 = ttest_ind(uni_r_houses_by_state['2008q4'], houses_r_by_state['2008q4'])
    p3 = ttest_ind(uni_r_houses_by_state['2009q1'], houses_r_by_state['2009q1'])
    p4 = ttest_ind(uni_r_houses_by_state['2009q2'], houses_r_by_state['2009q2'])
    average_p = ((p1[1] + p2[1] + p3[1] + p4[1]) / 4)
    return (False, ((p1[1] + p2[1] + p3[1] + p4[1]) / 4), "non-university town")
run_ttest()

/Users/nicholas.corneau/anaconda/envs/data-science/lib/python3.6/site-packages/ipykernel/__main__.py:29: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


(False, 9.833903597139958e-06, 'non-university town')

## To conclude, University towns are just as affected as regular towns during recessions.